In [1]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# =================== CNN Model ===================
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # -> [B, 32, 32, 32]
        x = self.pool(torch.relu(self.conv2(x)))  # -> [B, 64, 16, 16]
        x = x.view(-1, 64 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# =================== Dataset ===================
class SignLanguageDataset(Dataset):
    def __init__(self, json_path, image_folder, image_size=(64, 64)):
        with open(json_path, "r", encoding="utf-8") as f:
            self.data = json.load(f)

        self.images = []
        self.labels = []
        self.label_map = {word: idx for idx, word in enumerate(self.data.keys())}

        for word, frames in self.data.items():
            for frame in frames:
                img_path = os.path.join(image_folder, os.path.basename(frame["image_path"]))
                if os.path.exists(img_path):
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = cv2.resize(img, image_size)
                        img = img / 255.0
                        self.images.append(img)
                        self.labels.append(self.label_map[word])

        self.images = np.array(self.images, dtype=np.float32)
        self.labels = np.array(self.labels, dtype=np.int64)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        img = np.transpose(img, (2, 0, 1))  # [C, H, W]
        return torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# =================== Eğitim Fonksiyonu ===================
def train_model(json_path, image_folder, model_save_path, epochs=10, batch_size=32, lr=0.001):
    dataset = SignLanguageDataset(json_path, image_folder)
    num_classes = len(dataset.label_map)

    # Veriyi eğitim ve doğrulama olarak ayır
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=0.2, stratify=dataset.labels)
    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    val_dataset = torch.utils.data.Subset(dataset, val_idx)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    model = CNNModel(num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        val_loss, val_acc = 0.0, 0.0
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
            val_acc = correct / total

        print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    torch.save(model.state_dict(), model_save_path)
    print(f"\n✅ Eğitim tamamlandı. Model kaydedildi: {model_save_path}")

# =================== Çalıştır ===================
if __name__ == "__main__":
    json_path = r"C:\Users\hp\Desktop\yeniyazilim\output_data (1).json"
    image_folder = r"C:\Users\hp\Desktop\yeniyazilim\OutputImages"
    model_save_path = r"C:\Users\hp\Desktop\yeniyazilim\ann_sign_language_model.pth"

    train_model(json_path, image_folder, model_save_path, epochs=10)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.